In [4]:
import pandas as pd
import os
from dotenv import load_dotenv
import googlemaps
from geopy import distance
import folium

In [5]:
data = pd.read_csv('lista_final.csv',sep='|')

In [5]:
load_dotenv()

True

In [6]:
API_key=os.getenv('API')

In [7]:
gmaps_key = googlemaps.Client(key=API_key)

In [8]:
def get_coord(string):
    geocode_result = gmaps_key.geocode(string)
    try:
        lat = geocode_result[0]['geometry']['location']['lat']
        lon = geocode_result[0]['geometry']['location']['lng']
        return [lat,lon]
    except:
        return [None,None]

In [12]:
def get_coord(string):
    geocode_result = data['Distance']
    #try:
        #lat = geocode_result[0]['geometry']['location']['lat']
        #lon = geocode_result[0]['geometry']['location']['lng']
        #return [lat,lon]
    #except:
        #return [None,None]

In [10]:
data['Rango precios'] = data['Price(MX$)']

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 9 columns):
Address          588 non-null object
Price(MX$)       588 non-null int64
Bedrooms         588 non-null int64
Bathrooms        579 non-null float64
Size(m2)         588 non-null object
Latitud          588 non-null float64
Longitud         588 non-null float64
Coordinates      588 non-null object
Rango precios    588 non-null int64
dtypes: float64(3), int64(3), object(3)
memory usage: 41.4+ KB


In [13]:
def rentas():
    rango_precios = {'1':'5000-10000','2':'10000-15000','3':'15000-20000','4':'20000-25000','5':'25000-30000','6':'30000-35000','7':'35000-40000','8':'40000-45000','9':'45000-50000','10':'>50000'}
    dummy = data.copy()
    
    indice_precios = rango_precios[input(f'Selecciona un rango de precios: {str(rango_precios)}')].split('-')
    
    dummy = dummy[dummy['Price(MX$)'].between(int(indice_precios[0]), int(indice_precios[1]))]
    
    indice_hab = int(input((f'Selecciona número de habitantes: ')))
    dummy = dummy[dummy['Bedrooms'] == indice_hab]

    coord_trab = input(f'Cual es tu ubicacion? : ')

    ubicacion = get_coord(coord_trab)
    dummy['Distance'] = dummy['Coordinates'].str.split(',').apply(\
                        lambda x:get_distance(x, ubicacion))
    
    dummy = dummy[dummy['Distance']<=5]
    
    return dummy, ubicacion

In [14]:
test, ubi = rentas()

Selecciona un rango de precios: {'1': '5000-10000', '2': '10000-15000', '3': '15000-20000', '4': '20000-25000', '5': '25000-30000', '6': '30000-35000', '7': '35000-40000', '8': '40000-45000', '9': '45000-50000', '10': '>50000'}1
Selecciona número de habitantes: 3
Cual es tu ubicacion? : tonala 10


KeyError: 'Distance'

In [243]:
ubi

[19.4212752, -99.1631294]

In [242]:
test['Address']

38       Centro (área 1), Cuauhtémoc, Distrito Federa...
45       Centro (área 1), Cuauhtémoc, Distrito Federa...
52       Centro (área 1), Cuauhtémoc, Distrito Federa...
60       Centro (área 1), Cuauhtémoc, Distrito Federa...
67       Centro (área 1), Cuauhtémoc, Distrito Federa...
74       Centro (área 1), Cuauhtémoc, Distrito Federa...
84       Centro (área 1), Cuauhtémoc, Distrito Federa...
137      Golfo De San Jorge 37 Tacuba Miguel Hidalgo,...
152      Guanábana 10, Nueva Santa Maria, Azcapotzalc...
156      Lago Zirahuen 98, Colonia Anahuac I Sección,...
186      R.salido 166, Independencia, Benito Juárez, ...
290      Rafael Solana 5, Independencia, Benito Juáre...
296      Thomas A Edison 139, San Rafael, Cuauhtémoc,...
310      11 De Abril 177 303 A, Ciudad De México, Dis...
338      Peral 26, Santa Maria La Ribera, Cuauhtémoc,...
388      Av Cuauhtemoc 514, Narvarte, Benito Juárez, ...
481      Lago Como 128, 504, Miguel Hidalgo, Distrito...
537      Avenida 2 200, San Ped

In [159]:
depas = rentas()

Selecciona un rango de precios: {'1': '5000-10000', '2': '10000-15000', '3': '15000-20000', '4': '20000-25000', '5': '25000-30000', '6': '30000-35000', '7': '35000-40000', '8': '40000-45000', '9': '45000-50000', '10': '>50000'}1
Selecciona número de habitantes: {'1': '1', '2': '2', '3': '3', '4': '4', '5': '>=5'}3
Selecciona la alcaldía en donde trabajes: {'1': 'Álvaro Obregón', '2': 'Azcapotzalco', '3': 'Benito Juárez', '4': 'Coyoacán', '5': 'Cuajimalpa', '6': 'Cuauhtémoc', '7': 'Gustavo A.Madero', '8': 'Iztacalco', '9': 'Iztapalapa', '10': 'Magdalena Contreras', '11': 'Miguel Hidalgo', '12': 'Milpa Alta', '13': 'Tláhuac', '14': 'Tlalpan', '15': 'Venustiano Carranza', '16': 'Xochimilco'}tonala 10
Selecciona un hobby: {'1': 'Ejercio', '2': 'Pintar', '3': 'Leer', '4': 'Casinos', '5': 'Museos', '6': 'Comer', '7': 'Cine', '8': 'Bar'}2
{'rango': '5000-10000', 'Número Habitantes': '3', 'Lugar de trabajo': None, 'Hobby': 'Pintar'}


In [203]:
def get_distance(pointx, pointy):
    return distance.distance(pointx,pointy)

In [204]:
get_distance([-39, 99], [-40, 98])

Distance(140.44294774014736)

In [142]:
import operator
def get_rooms(coords,precio,rooms):
        return distance.distance(coords,precio,rooms, depas['Lugar de trabajo'],depas['rango_precios'],depas['numero_habitantes'])

In [193]:
#Calcular coordenadas
data['Distance'] = data['Coordinates'].apply(lambda x: get_distance(x.split(",")))

In [194]:
data.sort_values(by='Distance').head()

,Address,Price(MX$),Bedrooms,Bathrooms,Size(m2),Latitud,Longitud,Coordinates,Distance
146,"Avenida San Miguel 40, Xalpa, Ciudad De México",8500,2,2.0,100,19.332842,-99.032084,"19.332842000000003,-99.0320843",10965.546654626944 km
248,"7 (periferico Oriental), Iztacalco, Distrito...",6000,2,1.0,45,19.397102,-99.058471,"19.3971015,-99.0584708",10967.927781939256 km
583,"Josafat F. Márquez 26, Colonial Iztapalapa, ...",7500,2,2.0,78,19.344689,-99.057305,"19.3446888,-99.0573055",10968.12167274313 km
554,"Calle 4 Num 308, Agrícola Pantitlán, Iztacalco",4500,2,1.0,60,19.415524,-99.062349,"19.4155236,-99.0623488",10968.223156256057 km
516,"Calle 3, Agrícola Pantitlán, Iztacalco",3500,1,1.0,35,19.413731,-99.064482,"19.413731100000003,-99.0644816",10968.457667843133 km


In [134]:
#data.to_csv('Final_distances.csv', sep="|", index=False)

In [225]:
mapa = folium.Map(
    location=depas['Lugar de trabajo'],
    zoom_start=15,
    tiles='openstreetmap')
tooltip = 'Click me!'
mapa

In [226]:
for index, row in data.head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=500,
                  popup=row['Address'],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa)

In [227]:
mapa

In [228]:
mapa2 = folium.Map(
    location=depas['Lugar de trabajo'],
    zoom_start=15,
    tiles='openstreetmap')
tooltip = 'Click me!'


for index, row in data.sort_values(by='Distance').head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=15,
                  popup=row[['Address','Bedrooms','Price(MX$)']],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa2)


In [244]:
ubi

[19.4212752, -99.1631294]

In [247]:
mapa2 = folium.Map(
    location=depas['Lugar de trabajo'],
    zoom_start=15,
    tiles='openstreetmap')
tooltip = 'Click me!'


for index, row in test.sort_values(by='Distance').head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=500,
                  popup=row[['Address','Bedrooms','Price(MX$)']],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa2)
    
folium.Marker(ubi, icon=folium.Icon(color='blue',icon='home')).add_to(mapa2)


In [248]:
mapa2

In [108]:
data[data['Bedrooms']==2]
data[data['Price(MX$)'] == 50000]

,Address,Price(MX$),Bedrooms,Bathrooms,Size(m2),Latitud,Longitud,Coordinates,Distance
117,"Presa Falcón 243, Ph1, Ampliación Granada, M...",50000,2,3.0,220,19.443572,-99.204573,"19.4435717,-99.204573",10982.96563934168 km
257,Departamento En Renta En Carrimon De 3 Recam...,50000,3,3.0,290,19.385382,-99.254108,"19.3853815,-99.2541083",10988.518494836617 km
389,"Campos Elíseos 1, Polanco, Miguel Hidalgo, D...",50000,3,3.0,274,19.428787,-99.180368,"19.4287872,-99.1803683",10980.518474211114 km
579,"Mazaryk 400, Polanco, Miguel Hidalgo, Distri...",50000,4,5.0,350,19.431703,-99.198464,"19.4317034,-99.198464",10982.398041889843 km
